## Test of accelerated Monte Carlo ##

In [2]:
import sys
sys.path.insert(0, "../python/")

from particle import particle
from physics import em_physics
from cylinder import cylinder

import matplotlib.pyplot as plt
import numpy as np
from numba import jit
import pandas as pd

import json
import uproot
import csv

ModuleNotFoundError: No module named 'particle'

### Define the Geometry ###

In [13]:
# define the geometry
radius = 50 # cm
height = 100 #cm
cryostat = cylinder(R=radius,h=height)

radius = 40 # cm
height = 80 #cm
fiducial = cylinder(R=radius,h=height)

cylinder::__init__ Define cylinder with R= 50  and height= 100
cylinder::__init__ Define cylinder with R= 40  and height= 80


### Setup the photon physics ###

In [14]:
# define the physics
em = em_physics()

### Event generation ###

In [100]:
#%%prun -D my.profile
# number of events
#
nevent = 10000001
#
# gamma energy
#
energy = 1000 # keV
#
# output filename
#
write_mode = 'json'
#write_mode = 'txt'
#write_mode = 'csv'


mcout = '../mcdata/testdata' + '.' + write_mode 


#
# open output data file
#
f = open(mcout,'w')
if write_mode == 'json':
    f.write('[\n')  
#
# event generation loop
#
for ieve in range(nevent):
    
    if ieve%25000 == 0:
        print("generated ",ieve," events")
    #
    # make a particle
    #
    p = particle(type='gamma',
                 energy=energy, 
                 geometry=cryostat, 
                 fiducial=fiducial, 
                 vrt='fiducial_scatter',
                 edep_max=250,
                 nscatter_max=4,
                 physics=em,
                 debug=False)
    
    #
    # propagate the particle and retrieve the intersection points
    #
    p.propagate()

    #
    # store data
    #
    if write_mode == 'txt':
        
        wstring = "{:d} {:d} {:7.6e} {:7.6f} {:7.6f} {:7.6f} {:7.6f} ".format(ieve,len(p.xint), 
                                                                                p.weight,p.edep,
                                                                                p.x0start[0],p.x0start[1],p.x0start[2])
        f.write(wstring)
         
    if write_mode == 'csv':   
        wstring = "{:d}, {:d}, {:7.6e}, {:7.6f}, {:7.6f}, {:7.6f}, {:7.6f}".format(ieve,len(p.xint), 
                                                                                p.weight,p.edep,
                                                                                p.x0start[0],p.x0start[1],p.x0start[2])
        f.write(wstring)
        
            
            
    
    for i in range(len(p.xint)):
        #
        # data structure for each energy deposit
        #
        if write_mode == 'json':
            dd = {'x':p.xint[i][0][0],'y':p.xint[i][0][1],'z':p.xint[i][0][2],'de':p.xint[i][1],
                  'w':p.weight,'n':p.nscatter}
            json.dump(dd,f, sort_keys=True, indent=8)
            f.write(',\n')
            
        else:
            wstring = "{:7.6f}, {:7.6f}, {:7.6f}, {:7.6f},".format(p.xint[i][0][0],
                                                                  p.xint[i][0][1],
                                                                  p.xint[i][0][2],
                                                                  p.xint[i][1])
            
            f.write(wstring)
           
              
    f.write("\n") 
#
# close output data file
#
if write_mode == 'json':
    f.write('{}]')
elif write_mode == 'txt':
    f.write('-1\n')

    
f.close()

generated  0  events
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8628297990986  keV dE_max = -0.13717020090143706  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8825480236183  keV dE_max = -0.11745197638174432  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.820000699326  keV dE_max = -0.17999930067401237  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9240857153039  keV dE_max = -0.07591428469606853  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8568736339766  keV dE_max = -0.1431263660233526  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9958956074403  keV dE_max = -0.004104392559725056  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8350501591625  keV dE_max = -0.16494984083749387  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.6150545422158  keV dE_max = -0.3849454577841698  keV
physics::calculate_cost_min ERROR Bad energy 

physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9583746255738  keV dE_max = -0.04162537442618941  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8609472462886  keV dE_max = -0.13905275371143944  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9564271875315  keV dE_max = -0.04357281246848288  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9224738617833  keV dE_max = -0.07752613821674004  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.955439288065  keV dE_max = -0.04456071193499156  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9030148462574  keV dE_max = -0.09698515374259387  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9511771973093  keV dE_max = -0.04882280269066541  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8348809409389  keV dE_max = -0.16511905906111224  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.88

physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8495362537524  keV dE_max = -0.15046374624762393  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8495362537524  keV dE_max = -0.15046374624762393  keV
generated  175000  events
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.7866275075927  keV dE_max = -0.21337249240730216  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.858540902298  keV dE_max = -0.1414590977019543  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8898501648854  keV dE_max = -0.11014983511461196  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8365634706834  keV dE_max = -0.1634365293166411  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8365634706834  keV dE_max = -0.1634365293166411  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8699973012833  keV dE_max = -0.13000269871668024  keV
physics::calculate_cost_min ERROR Bad ener

physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9442832371151  keV dE_max = -0.055716762884912896  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9548300510755  keV dE_max = -0.04516994892446746  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.7993631838033  keV dE_max = -0.20063681619672025  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.949722195521  keV dE_max = -0.05027780447903751  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9041236138625  keV dE_max = -0.09587638613754734  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9519946175639  keV dE_max = -0.04800538243614483  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8747323569038  keV dE_max = -0.1252676430962083  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9547791805601  keV dE_max = -0.04522081943991907  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.99

physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9856542085208  keV dE_max = -0.014345791479172476  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9104748313324  keV dE_max = -0.08952516866759197  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9618676840912  keV dE_max = -0.038132315908796954  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8557466676531  keV dE_max = -0.14425333234692062  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9301430533212  keV dE_max = -0.06985694667878306  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.7100916420346  keV dE_max = -0.2899083579653734  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9801378939873  keV dE_max = -0.019862106012737968  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8784395794826  keV dE_max = -0.12156042051742588  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749

...

physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8261647775608  keV dE_max = -0.17383522243915195  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.891131029891  keV dE_max = -0.10886897010902885  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9250275240964  keV dE_max = -0.07497247590356437  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.7933011579978  keV dE_max = -0.20669884200219713  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9426478352817  keV dE_max = -0.05735216471828153  keV
generated  9700000  events
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8909726752058  keV dE_max = -0.10902732479416954  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8742818822113  keV dE_max = -0.12571811778866504  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9765357593828  keV dE_max = -0.023464240617158794  keV
physics::calculate_cost_min ERROR Bad

physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9696397145334  keV dE_max = -0.030360285466599635  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9696397145334  keV dE_max = -0.030360285466599635  keV
particle::propagate ERROR.... bad intersection. discard event.
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8953317210778  keV dE_max = -0.10466827892219044  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.7830630143847  keV dE_max = -0.21693698561534802  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.881367988691  keV dE_max = -0.11863201130904599  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8592388384701  keV dE_max = -0.1407611615298947  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8442134427978  keV dE_max = -0.15578655720219103  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.7878158861784  keV dE_max = -0.2121841138216496  keV
ph

physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9226968833283  keV dE_max = -0.07730311667171463  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9725598433832  keV dE_max = -0.02744015661676258  keV
generated  9825000  events
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.7927495155734  keV dE_max = -0.207250484426595  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8954879656322  keV dE_max = -0.10451203436775813  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8210291006137  keV dE_max = -0.17897089938628596  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8485080215098  keV dE_max = -0.15149197849018492  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8199756937635  keV dE_max = -0.18002430623653254  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9215869358627  keV dE_max = -0.07841306413729399  keV
physics::calculate_cost_min ERROR Bad e

physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.795902506852  keV dE_max = -0.20409749314796954  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.5759298571554  keV dE_max = -0.42407014284458455  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9554360268569  keV dE_max = -0.04456397314311289  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9554360268569  keV dE_max = -0.04456397314311289  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8425650333204  keV dE_max = -0.15743496667960244  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8083978732881  keV dE_max = -0.19160212671192767  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8083978732881  keV dE_max = -0.19160212671192767  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8381725861711  keV dE_max = -0.1618274138288598  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.838

physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9291968015041  keV dE_max = -0.07080319849592343  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.957756540099  keV dE_max = -0.042243459900987546  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.927854993123  keV dE_max = -0.0721450068770082  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8895830853342  keV dE_max = -0.1104169146658478  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8613582919195  keV dE_max = -0.13864170808051313  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.9207158846227  keV dE_max = -0.07928411537727698  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.938234838454  keV dE_max = -0.061765161545963565  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.8376765451891  keV dE_max = -0.1623234548109167  keV
physics::calculate_cost_min ERROR Bad energy or de_max. E= 749.81262

In [ ]:
!snakeviz my.profile

snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2FUsers%2Folivier%2FPycharmProjects%2FFastMC%2Fnotebooks%2Fmy.profile
